In [20]:
import torch
import threading
import torch.nn as nn
from transformers import AutoModelForCausalLM, AutoTokenizer

class Foundation:
    def __init__(self):
        # Loading DialoGPT-Medium
        self.ltm = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")
        self.tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
        
    def generate_text(self, prompt):
        # ENcode the prompt and generate a response
        inputs = self.tokenizer.encode(prompt + self.tokenizer.eos_token, return_tensors="pt")
        response = self.ltm.generate(inputs, max_length=1000, pad_token_id=self.tokenizer.eos_token_id)
        return self.tokenizer.decode(response[0], skip_special_tokens=True)
    
    def process_input(self, input_data):
        # Generate text based on the input
        prompt = f"Memory query: {input_data}"
        return self.generate_text(prompt)

class MemoryManager:
    def __init__(self, stm_capacity=5):
        self.stm_capacity = stm_capacity # Limit for short-term memory
        self.stm_head = None
        self.stm_size = 0
        self.ltm = Foundation() # Long-term memory storage
        self.mutex = threading.Lock() # Mutex for seamless access
        self.adaptive_weights = {} # Adaptive learning storage
    

class STM_Level_One(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(STM_Level_One, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self,x):
        # Initialize hidden state
        h_zero = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        # Forward propagate RNN
        out, _ = self.rnn(x, h_zero)
        # Decode the output for the last time step
        out = self.fc(out[:, -1, :])
        return out

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    def forward(self, x):
        lstm_out,_=self.lstm(x)
        out = self.fc(lstm_out[:, -1, :]) # Get last time-step output
        return out
    
    # Hybrid LSTM + SNN Integration
    def hybrid_model_step(model, snn_layer, memory_manager, optimizer, criterion, memory_sequence, num_epochs=10):
        # Convert memory to numerical data by assigning each letter to number
        letter_to_index = {letter: idx for idx, letter in enumerate('abcdefghijklmnopqrstuvwxyz')}
        index_to_letter = {idx: letter for idx, letter in enumerate('abcdefghijklmnopqrstuvwxyz')}
        
        # Reshape input for LSTM
        inputs = torch.Tensor(data).view(1, -1, 1)
        targets = torch.tensor([target], dtype=torch.long) # Ensure 1D tensor with integer class index
        
        model.train()
        optimizer.zero_grad()
        
        # LSTM forward pass
        output = model(inputs)
        loss = criterion(output, targets)
        loss.backward()
        optimizer.step()
        
        
class SNNLayer(nn.Module):
    def __init__(self, input_size, num_neurons, threshold=-1.0, decay=0.9):
        super(SNNLayer, self).__init__()
        self.num_neurons = num_neurons
        self.input_size = input_size
        self.threshold = threshold
        self.decay = decay
        self.neuron_potentials = torch.zeros(num_neurons)
        self.neuron_spikes = torch.zeros(num_neurons)
        
        # Synaptic weights (connections between input and neurons)
        self.synaptic_weights = nn.Parameter(torch.randn(input_size, num_neurons))
        
    def forward(self, x):
        x = x.view(-1, self.input_size) # Reshape to match syn_shape
        self.neuron_potentials *= self.decay # Decay the neuron potentials over time
        self.neuron_potentials += torch.matmul(x, self.synaptic_weights).squeeze(0)
        
        # Check if neurons fire (spike) based on threshold
        self.neuron_spikes = (self.neuron_potentials >= self.threshold).float()
        self.neuron_potentials *= (1-self.neuron_spikes)
        
        return self.neuron_spikes

    
 # Step 2: Initialize Models
foundation = Foundation()
memory_manager = MemoryManager()
snn_layer = SNNLayer(input_size=26, num_neurons=5) # Adjust input_size to match your needs
lstm_model = LSTMModel(input_size=1, hidden_size=32, output_size=26)
# Optional: Sample STM
stm = STM_Level_One(input_size=1, hidden_size=16, num_layers=1, output_size=26)    

def neurolink(foundation, memory_manager, snn_layer, lstm_model, stm):
    """Connect these models together to establish neurolink returns output after passing through all of them"""
    user_input = input("You: ")
   
    # Step 3: Language -> Token
    generated_text = foundation.generate_text(user_input)
    print("\n[Foundation Output]: ", generated_text)
    
    # Step 4: Token-level encoding (example: basic character-level numerical encoding)
    letter_to_index = {ch: idx for idx, ch in enumerate('abcdefghijklmnopqrstuvwxyz')}
    data = [letter_to_index.get(ch.lower(), 0) for ch in generated_text if ch.lower() in letter_to_index]
    if len(data) == 0:
        return '[Error]: No valid data from input.'
    
    input_tensor = torch.Tensor(data).view(1, -1, 1)
    
    # Step 5: Process through LSTM
    lstm_output = lstm_model(input_tensor)
    print("[LSTM Output]:", lstm_output)
    
    # Step 6: SNN Layer step
    snn_output = snn_layer(lstm_output)
    print("[SNN Spikes]:", snn_output)
    
    return snn_output

start_input = input("Would you like to train Aurora?: ")

while start_input.lower() in ['yes', 'y', 'yeah', 'sure']:
    neurolink(foundation, memory_manager, snn_layer, lstm_model, stm)
    start_input = input("Would you like to continue training Aurora?: ")

    

Would you like to train Aurora?: Yes
You: how are you

[Foundation Output]:  how are youI'm good, how are you?
[LSTM Output]: tensor([[-0.0207, -0.3179,  0.1043, -0.0201,  0.2208, -0.1027, -0.0597, -0.1454,
         -0.3617, -0.0639, -0.2784,  0.0127,  0.0818,  0.0957,  0.3663,  0.3111,
          0.3488,  0.5083, -0.1074,  0.1005, -0.1142, -0.2859, -0.3738, -0.4298,
         -0.0197, -0.1975]], grad_fn=<AddmmBackward0>)
[SNN Spikes]: tensor([0., 1., 1., 1., 1.])
Would you like to continue training Aurora?: yes
You: ABCDEFG. What comes next?

[Foundation Output]:  ABCDEFG. What comes next?ABCDEGF. What comes next?
[LSTM Output]: tensor([[-0.0230, -0.3090,  0.0961, -0.0220,  0.2148, -0.0905, -0.0587, -0.1516,
         -0.3515, -0.0659, -0.2797,  0.0045,  0.0826,  0.1019,  0.3749,  0.3088,
          0.3404,  0.5005, -0.1010,  0.1036, -0.1085, -0.2725, -0.3708, -0.4283,
         -0.0177, -0.1934]], grad_fn=<AddmmBackward0>)
[SNN Spikes]: tensor([0., 1., 1., 1., 1.])
Would you like to conti

In [15]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

model_name = "microsoft/DialoGPT-medium"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)



In [34]:
information = {'a':1, 'b':2, 'c':3, 'd':4, 'e':5, 'f':6, 'g':7, 'h':8, 'i':9}

# Start weights and learninh counters
weights = {key: 0 for key in information}
learning_log = []

weight_limit = 100
x = 0

while x < weight_limit:
    user_input = input("Enter a lowercase letter a-i (or 'q' to quit): ")
    if user_input == 'q':
        x = 101
        
    if user_input in information:
        value = information[user_input]
        x += 1
        
        # Update weight based on learning rule (can be expanded)
        weights[user_input] += (value * x**2 + 1)
        learning_log.append(user_input)
    
    else:
        print("Invalid input. Try a letter between a and i.")
        
#Output the most learned information
most_learned = max(weights, key=weights.get)
print("\n--- Learning Summary ---")
print("Most reinforced info:", most_learned)
print("Weight:", weights[most_learned])
print("All weights:", weights)

Enter a lowercase letter a-i (or 'q' to quit): a
Enter a lowercase letter a-i (or 'q' to quit): a
Enter a lowercase letter a-i (or 'q' to quit): i
Enter a lowercase letter a-i (or 'q' to quit): iii
Invalid input. Try a letter between a and i.
Enter a lowercase letter a-i (or 'q' to quit): a
Enter a lowercase letter a-i (or 'q' to quit): a
Enter a lowercase letter a-i (or 'q' to quit): b
Enter a lowercase letter a-i (or 'q' to quit): c
Enter a lowercase letter a-i (or 'q' to quit): i
Enter a lowercase letter a-i (or 'q' to quit): b
Enter a lowercase letter a-i (or 'q' to quit): i
Enter a lowercase letter a-i (or 'q' to quit): a
Enter a lowercase letter a-i (or 'q' to quit): b
Enter a lowercase letter a-i (or 'q' to quit): a
Enter a lowercase letter a-i (or 'q' to quit): i
Enter a lowercase letter a-i (or 'q' to quit): c
Enter a lowercase letter a-i (or 'q' to quit): f
Enter a lowercase letter a-i (or 'q' to quit): f
Enter a lowercase letter a-i (or 'q' to quit): f
Enter a lowercase lett

In [50]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import GPT2Tokenizer
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
#---Configuration---
#tokens = [token for token in tokens if token < tokenizer.vocab_size]
vocab_size = tokenizer.vocab_size # Adjust based on your tokenized dataset
embedding_dim = 64
num_heads = 2
num_layers = 2
block_size = 32 # Sequence length
ffn_hidden = 128
#input = torch.randint(0, vocab_size, (batch_size, seq_length))

#--- Simple Transformer Block ---
class TransformerBlock(nn.Module):
    def __init__(self, embedding_dim, num_heads, ffn_hidden):
        super().__init__()
        self.attn = nn.MultiheadAttention(embedding_dim, num_heads, batch_first=True)
        self.ln1 = nn.LayerNorm(embedding_dim)
        self.ffn = nn.Sequential(nn.Linear(embedding_dim, ffn_hidden),
                                nn.ReLU(), nn.Linear(ffn_hidden, embedding_dim),)
        self.ln2 = nn.LayerNorm(embedding_dim)
    
    def forward(self, x):
        attn_output,_ = self.attn(x,x,x)
        x = self.ln1(x + attn_output)
        ffn_output = self.ffn(x)
        x = self.ln2(x + ffn_output)
        return x
    
class MemoryManager:
    def __init__(self, decay_factor=0.99, threshold=0.5):
        self.decay_factor = decay_factor # Decay factor for memory weight
        self.threshold = threshold # Cognitive score threshold for LTM promotion
        self.memory_units = [] # Store memory units (information)
        
    def update_weight(self, w_i):
        """Apply decay to the memory weight."""
        return w_i * self.decay_factor # Apply decay to weight
    
    def f(self,n):
        """Fibonacci-like recursive function for familiarity adjustment"""
        if n == 0 or n == 1:
            return 1
        return self.f(n-1) + self.f(n-2)
    
    def cognitive_score(self):
        """Calculate the cognitive score based on the weighted information, entropy, and Fibonacci adjustment."""
        
        score = 0
        for unit in self.memory_units:
            weight = self.update_weight(unit['weight'])
            entropy = unit['entropy']
            fibonacci = self.f(unit['N-1']) + self.f(unit['N-2']) # Recursive function for familiarity
            score += (2 ** unit['k']-1) * weight * entropy * fibonacci
            
        # Normalize by dividing by the number of memory units (or other normalization logic)
        score /= len(self.memory_units)
        
        # Decision: promote to LTM or retain STM
        if score >= self.threshold:
            return 'Promote to LTM'
        else:
            return 'Retain in STM'
        return score
    
    def add_memory_unit(self, weight, entropy, N, k):
        """Add a new memory unit to the memory system."""
        unit = {'weight': weight, 'entropy': entropy, 'N-1': N-1, 'N-2': N-2, 'k':k}
        self.memory_units.append(unit)
        
    def clear_memory(self):
        """Clear all memory units."""
        self.memory_units = []
        
# Example usage:

# Initialize memory manager
#memory_manager = MemoryManager(decay_factor=0.99, threshold=0.5)

#add some memory units to simulate information
#memory_manager.add_memory_unit(weight=0.8, entropy=0.2, N=3, k=1)
#memory_manager.add_memory_unit(weight=0.7, entropy=0.3, N=4, k=2)

# Compute the cognitive score and determine memory transfer
#status = memory_manager.cognitive_score()
#print(status)

class STM_Level_One(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(STM_Level_One, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self,x):
        # Initialize hidden state
        h_zero = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        # Forward propagate RNN
        out, _ = self.rnn(x, h_zero)
        # Decode the output for the last time step
        out = self.fc(out[:, -1, :])
        return out

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)
    def forward(self, x):
        lstm_out,_=self.lstm(x)
        out = self.fc(lstm_out[:, -1, :]) # Get last time-step output
        return out
    
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    
    # Hybrid LSTM + SNN Integration
    def hybrid_model_step(model, snn_layer, memory_manager, optimizer, criterion, memory_sequence, num_epochs=10):
        # Iterate over the memory sequence
        for epoch in range(num_epochs):
            for data in memory_sequence:
            # Tokenize the input sequence using the tokenizer
                inputs = tokenizer.encode(data, add_special_tokens=True)  # Tokenize text to numerical tokens
                # Skip too-short sequences
                if len(token_ids) < 2:
                    continue
                inputs = torch.tensor(inputs, dtype=torch.long).unsqueeze(0)  # Reshape for LSTM input
            
            # Use the last token as the target
                targets = torch.tensor([inputs[0, -1]], dtype=torch.long)  # The last token as the target
            
                model.train()
                optimizer.zero_grad()
            
            # LSTM forward pass
                lstm_output = model(inputs)
            
            # Calculate loss with the target
                loss = criterion(lstm_output, targets)
            
            # Backpropagate loss and optimize LSTM weights
                loss.backward()
                optimizer.step()
            
                #MemoryManager Interaction
                #Let's assume: entropy ~ loss.item(), weight = inverse of loss
                weight = max(1.0 - loss.item(), 0.0)
                entropy = loss.item()
            
                # Familiarity indicators (example: token length as a proxy)
                N = len(inputs)
                k = int(torch.argmax(lstm_output).item()) if lstm_output.ndim > 1 else int(lstm_output.item())
            
            # Update memory (using the memory manager)
                memory_manager.add_memory_unit(weight=weight, entropy=entropy, N=N, k=k)
            
            # Simulate spiking neural network behavior with SNN layer
                spikes = snn_layer(lstm_output)
            
            # Update the memory manager based on the results of the SNN spikes (if necessary)
                memory_manager.cognitive_score()

                print(f"Epoch {epoch + 1}, Loss: {loss.item()}")
                print(f"Spikes: {spikes}")
            
            # Evaluate memory promotion after each epoch
            decision = memory_manager.cognitive_score()
            print(f"Epoch {epoch+1} -> Cognitive Score Decision: {decision}")

        
class SNNLayer(nn.Module):
    def __init__(self, input_size, output_size, num_neurons, threshold=-1.0, decay=0.9):
        super(SNNLayer, self).__init__()
        self.num_neurons = num_neurons
        self.input_size = input_size
        self.output_size = output_size
        self.threshold = threshold
        self.decay = decay
        self.neuron_potentials = torch.zeros(num_neurons)
        self.neuron_spikes = torch.zeros(num_neurons)
        
        # Synaptic weights (connections between input and neurons)
        self.synaptic_weights = nn.Parameter(torch.randn(self.input_size, output_size))
        
    def forward(self, x):
        if x.size(1) != self.input_size:
            raise ValueError(f"Input size should be {self.input_size}, but got {x.size(1)}")
            
        x = x.view(-1,64) # Reshape to match syn_shape
        print(x.shape)  # Print the input shape
        print(self.synaptic_weights.shape)  # Print the weights shape

        self.neuron_potentials *= self.decay # Decay the neuron potentials over time
        self.neuron_potentials += torch.matmul(x, self.synaptic_weights).squeeze(1)
        
        # Check if neurons fire (spike) based on threshold
        self.neuron_spikes = (self.neuron_potentials >= self.threshold).float()
        self.neuron_potentials *= (1-self.neuron_spikes)
        
        return self.neuron_spikes

#tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
vocab_size = tokenizer.vocab_size
store_memory = []

def memory_storer(input_text):
    tokens = tokenizer.encode(input_text, add_special_tokens=True)
    store_memory.append(tokens)
    return store_memory

class TextDataset(Dataset):
    def __init__(self,data):
        self.data = data
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        # Tokenize the text data at idx from the dataset
        input_text = self.data[idx]
        
        # Convert the tokenized input into a Pytorch Tensor
        #tensor_input = torch.tensor(tokenized_input)
        
        return torch.tensor(input_text) # Return the tokenized tensor
    
# --- Tiny GPT Model ---
class AuroraGPT(nn.Module):
    def __init__(self, vocab_size, embedding_dim, num_heads, num_layers, block_size, ffn_hidden):
        super().__init__()
        self.vocab_size = tokenizer.vocab_size
        self.token_embedding = nn.Embedding(tokenizer.vocab_size, embedding_dim)
        self.position_embedding = nn.Embedding(block_size, embedding_dim)
        self.blocks = nn.Sequential(*[TransformerBlock(embedding_dim, num_heads, ffn_hidden) for _ in range(num_layers)])
        self.ln_f = nn.LayerNorm(embedding_dim)
        self.head = nn.Linear(embedding_dim, vocab_size)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.to(self.device)
        
    def forward(self, idx):
        B, T = idx.shape
        tok_emb = self.token_embedding(idx)
        pos = torch.arange(T, device=idx.device).unsqueeze(0)
        pos_emb = self.position_embedding(pos)
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.head(x)

        return logits

    
class MetricsLogger:
    def __init__(self):
        self.losses = []
        self.entropies = []
        self.weights = []
        self.cognitive_scores = []
        self.ltm_promotions = []
    
    def log_step(self, loss, output_tensor, manager, N, k):
        self.losses.append(loss.item())
        
        # Simulated entropy and weight proxies
        entropy = output_tensor.std().item()
        weight = output_tensor.abs().mean().item()
        
        self.entropies.append(entropy)
        self.weights.append(weight)
        
        # Memory management check
        manager.add_memory_unit(weight, entropy, N=N, k=k)
        decision = manager.cognitive_score()
        self.cognitive_scores.append(manager.cognitive_score)
        
        if decision == 'Promote to LTM':
            self.ltm_promotions.append(0)
        else:
            self.ltm_promotions.append(0)
    
    def plot_all(self, model_name="AuroraGPT"):
        epochs = list(range(len(self.losses)))
        
        fig, axs = plt.subplots(2, 2, figsize=(14,10))
        fig.suptitle(f"Training Metrics for {model_name}", fontsize=16)
        
        # Loss over epochs
        axs[0,0].plot(epochs, self.losses, label="Loss")
        axs[0,0].set_title("Loss Over Epochs")
        axs[0,0].set_xlabel("Epoch")
        axs[0,0].set_ylabel("Loss")
        axs[0,0].legend()
        
        # LTM promotions count
        ltm_total = sum(self.ltm_promotions)
        axs[0,1].bar(["Promoted to LTM", "Retained in STM"],[ltm_total, len(self.ltm_promotions) - ltm_total], color=['green', 'orange'])
        axs[0,1].set_title("Memory Promotion Decisions")
        
        # Cognitive score histogram
        axs[1,0].hist(self.cognitive_scores, bins=10, color='blue', alpha=0.7)
        axs[1,0].set_title("Cognitive Score Distribution")
        axs[1,0].set_xlabel("Score")
        axs[1,0].set_ylabel("Frequency")
        
        # Entropy vs. Weight scatter
        axs[1,1].scatter(self.entropies, self.weights, alpha=0.5, color='purple')
        axs[1,1].set_title("Entropy vs. Weight")
        axs[1,1].set_xlabel("Entropy (std of output)")
        axs[1,1].set_ylabel("Avg. Output Weight")
        
        plt.tight_layout(rect=[0,0.03,1,0.95])
        plt.show()
    

model = AuroraGPT(vocab_size=tokenizer.vocab_size, embedding_dim=embedding_dim, block_size=block_size, num_layers=num_layers, num_heads=num_heads, ffn_hidden=ffn_hidden)
def real_time_training(model, snn_layer, memory_manager, optimizer, criterion, max_inputs=15, batch_size=5):
    logger = MetricsLogger() 
    num_inputs = 0
    while num_inputs < max_inputs:
        user_input = input("Type something ('q' to quit): ")
        
        # Exit condition if the user types 'q'
        if user_input == 'q':
            print("Training has stopped")
            break
        
        # Store memory and process input text
        memory_storer(user_input)
        num_inputs += 1
        
        # Train after collecting batch_size inputs
        if num_inputs % batch_size == 0:
            # Prepare batch data for training
            dataset = TextDataset(store_memory)
            dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
            
            # Training loop over the batch
            model.train()
            for data in dataloader:
                inputs = data
                inputs = inputs.to(model.device) # Ensure input is on the correct device
                
                print("Input shape: ", inputs.shape)
                print("Max token ID: ", inputs.max().item(), " | Vocab size:", model.vocab_size)
                print("Min token ID:", inputs.min().item())
                
                # Pass through AuroraGPT (tokenization and Transformer)
                logits = model(inputs)
                
                # LSTM step
                lstm_output = model(inputs) # could be same model or a seperate LSTM model
                
                # Calculate loss for the LSTM output
                print("Logits shape:", lstm_output.shape)
                print("Targets shape:", inputs.squeeze().shape)
                lstm_output = lstm_output[:, :, :64] # Take only the first 64 features if LSTM size is too big
                lstm_output = lstm_output.view(-1,64)
                target = inputs.squeeze()
                print("Target values:", target)
                target = torch.clamp(target, 0, 63)
                loss = criterion(lstm_output, target) # For simplicity, using inputs as targets
                
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                
                #Log metrics
                logger.log_step(loss, lstm_output, memory_manager, N=num_inputs, k=2)
                
                # Update memory based on the LSTM output (as a simulated step)
                memory_manager.add_memory_unit(weight=0.5, entropy=0.3, N=5, k=2)
                
                # Simulate SNN layer with the LSTM output
                spikes = snn_layer(lstm_output)
                
                # Update the memory manager after SNN simulation
                memory_manager.cognitive_score()
                
                print(f"Epoch {num_inputs // batch_size}, Loss: {loss.item()}")
                print(f"Spikes: {spikes}")
                
    print("Maximum number of inputs reached or stopped by user. Training complete.")
    logger.plot_all(model_name="AuroraGPT Alpha")
    
# Initialize all components:
#tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
memory_manager = MemoryManager(decay_factor=0.99, threshold=0.5)
model = AuroraGPT(vocab_size=100, embedding_dim=64, num_heads=2, num_layers=2, block_size=32, ffn_hidden=128)
snn_layer = SNNLayer(input_size=embedding_dim, output_size=block_size, num_neurons=50)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# Run real-time training
real_time_training(model, snn_layer, memory_manager, optimizer, criterion)

Type something ('q' to quit): a
Type something ('q' to quit): b
Type something ('q' to quit): c
Type something ('q' to quit): d
Type something ('q' to quit): e
Input shape:  torch.Size([5, 1])
Max token ID:  68  | Vocab size: 50257
Min token ID: 64
Logits shape: torch.Size([5, 1, 100])
Targets shape: torch.Size([5])
Target values: tensor([67, 64, 66, 68, 65])
torch.Size([5, 64])
torch.Size([64, 32])


RuntimeError: The size of tensor a (50) must match the size of tensor b (32) at non-singleton dimension 1

In [1]:
# ---Example usage---
model = AuroraGPT(vocab_size=vocab_size,
                 embedding_dim=embedding_dim,
                 num_heads=num_heads,
                 num_layers=num_layers,
                 block_size=block_size,
                 ffn_hidden=ffn_hidden)

# Random input for testing
input_ids = torch.randint(0, vocab_size, (2, block_size))
logits = model(input_ids)
print(logits.shape) # Should be (batch, sequence, vocab_size

x = 100
    
while x < 101:
    user_input = input("Type something ('q to quit'): ")
    memory_storer(user_input)
    
    if len(user_input) >= 25:
        print("Stored tokens: ", store_memory)
        x = 102
    
    elif user_input == 'q':
        print("Stored tokens: ", store_memory)
        x = 102
        
dataset = TextDataset(store_memory)

# Example: Iterate through the dataset to get the tokenized tensor data
for idx in range(len(dataset)):
    print(f"Sample{idx}: {dataset[idx]}")

NameError: name 'AuroraGPT' is not defined